In [13]:
## IMPORTS
%matplotlib inline
import collections
import glob
import time
import os
import easygui
import seaborn as sns
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler
from matplotlib import cm

warnings.filterwarnings('ignore') 

In [14]:
df = pd.read_csv("iris.csv")

In [15]:
dataset = df
for col in dataset.columns:
    if dataset[col].dtypes != 'float64':
        dataset = dataset.drop( columns = [col])

In [16]:
print(len(dataset))
print(len(dataset.columns))
print(dataset.columns)

150
4
Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width'], dtype='object')


In [1]:
# dist_names = ["alpha", "anglit", "arcsine", "argus", "beta", "betaprime", "bradford", "burr", "burr12", "cauchy", "chi",
#               "chi2","cosine","crystalball", "dgamma", "dweibull", "erlang", "expon", "exponnorm","exponweib", "exponpow", 
#                "f", "fatiguelife", "fisk", "foldcauchy", "foldnorm", "frechet_r", "frechet_l", "genlogistic", "gennorm", 
#                "genpareto", "genexpon", "genextreme","gausshyper", "gamma", "gengamma", "genhalflogistic", "gilbrat", 
#                "gompertz", "gumbel_r", "gumbel_l", "halfcauchy", "halflogistic", "halfnorm", "halfgennorm", "hypsecant", 
#                "invgamma", "invweibull", "johnsonsb", "johnsonsu", "kappa4", "kappa3", "ksone", "kstwobign", "laplace", 
#                "levy", "levy_l", "logistic", "loggamma", "loglaplace","lognorm", "loguniform", "lomax", "maxwell", "mielke", 
#                "moyal", "nakagami", "ncx2","ncf","norm", "pareto", "pearson3", "powerlaw", "powerlognorm", "powernorm","rdist", 
#                "rayleigh", "rice", "semicircular", "skewnorm", "t", "trapz", "triang","truncexpon", "uniform", "vonmises", 
#                "vonmises_line", "wald", "weibull_min", "weibull_max", "wrapcauchy"]

# print(len(dist_names))

91


In [18]:
def get_best_distribution(data):
    dist_names = ["alpha", "anglit", "arcsine", "argus", "beta", "betaprime", "bradford", "burr", "burr12", "cauchy", "chi",
                    "chi2","cosine","crystalball", "dgamma", "dweibull", "erlang", "expon", "exponnorm","exponweib",
                    "exponpow", "f", "fatiguelife", "fisk", "foldcauchy", "foldnorm", "frechet_r", "frechet_l", "genlogistic",
                    "gennorm", "genpareto", "genexpon", "genextreme", "gausshyper", "gamma", "gengamma", "genhalflogistic",
                    "geninvgauss", "gilbrat", "gompertz", "gumbel_r", "gumbel_l", "halfcauchy", "halflogistic", "halfnorm",
                    "halfgennorm", "hypsecant", "invgamma", "invgauss", "invweibull", "johnsonsb", "johnsonsu", "kappa4",
                    "kappa3", "ksone", "kstwobign", "laplace", "levy", "levy_l", "levy_stable", "logistic", "loggamma",
                    "loglaplace", "lognorm", "loguniform", "lomax", "maxwell", "mielke", "moyal", "nakagami", "ncx2",
                    "ncf", "nct", "norm", "norminvgauss", "pareto", "pearson3", "powerlaw", "powerlognorm", "powernorm",
                    "rdist", "rayleigh", "rice", "recipinvgauss", "semicircular", "skewnorm", "t", "trapz", "triang",
                    "truncexpon", "truncnorm", "tukeylambda", "uniform", "vonmises", "vonmises_line", "wald", "weibull_min",
                    "weibull_max", "wrapcauchy"]

    dist_time = [len(dist_names)*[],len(dist_names)*[]]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        start_time = time.time()
        dist = getattr(st, dist_name)
        param = dist.fit(data)
        params[dist_name] = param
        D, p = st.kstest(data, dist_name, args=param)
        dist_results.append((dist_name, p))
        dist_time[0].append(dist_name)
        dist_time[1].append(time.time() - start_time)
        

    # select the best fitted distribution
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    return best_dist, best_p, params[best_dist],dist_time

In [19]:
time_for = len(dataset.columns)*[]
for col in dataset.columns:
    _,_,_,dist_time = get_best_distribution(dataset[col])
    time_for.append(dist_time)

In [29]:
e = { "Distributions":time_for[2][0], "Time":time_for[2][1]}
df_time2 = pd.DataFrame(e, columns=["Distributions", "Time"])
df_sorted = df_time2.sort_values(by=["Time"])
print(df_sorted.tail(30))

    Distributions        Time
72            nct    0.193224
47       invgamma    0.204641
20       exponpow    0.208931
79      powernorm    0.211919
49     invweibull    0.217333
30      genpareto    0.219532
87          trapz    0.224107
55      kstwobign    0.237732
48       invgauss    0.240683
35       gengamma    0.263220
45    halfgennorm    0.275204
74   norminvgauss    0.319558
7            burr    0.322369
19      exponweib    0.361094
39       gompertz    0.363574
23           fisk    0.387747
70           ncx2    0.475183
51      johnsonsu    0.500389
78   powerlognorm    0.503313
50      johnsonsb    0.544642
52         kappa4    0.643814
94  vonmises_line    0.662657
71            ncf    0.718334
93       vonmises    0.895089
33     gausshyper    1.411482
3           argus    1.487158
31       genexpon    2.772586
83  recipinvgauss    3.164300
37    geninvgauss    6.503833
59    levy_stable  140.490541


In [47]:
# time_for = len(dataset.columns)*[]
# for col in dataset.columns:
#     _,_,_,dist_time = get_best_distribution(dataset[col])
#     time_for.append(dist_time)

In [48]:
# for i,col in zip(range(len(time_for)),dataset.columns):
#     e = { "Distributions":time_for[i][0], "Time":time_for[i][1]}
#     df_time2 = pd.DataFrame(e, columns=["Distributions", "Time"])
#     df_sorted = df_time2.sort_values(by=["Time"])
#     df_sorted.to_csv("TimeDistColumn/time_"+col+".csv", index = False)

-----------------------------------------------

In [7]:
def get_best_distribution(data):
    dist_names = ["alpha", "anglit", "arcsine", "argus", "beta", "betaprime", "bradford", "burr", "burr12", 
                  "cauchy", "chi","chi2","cosine","crystalball", "dgamma", "dweibull", "erlang", "expon", 
                  "exponnorm","exponweib", "exponpow", "f", "fatiguelife", "fisk", "foldcauchy", "foldnorm", 
                  "frechet_r", "frechet_l", "genlogistic", "gennorm", "genpareto", "genexpon", "genextreme",
                  "gausshyper", "gamma", "gengamma", "genhalflogistic", "gilbrat", "gompertz", 
                  "gumbel_r", "gumbel_l", "halfcauchy", "halflogistic", "halfnorm", "halfgennorm", "hypsecant", 
                  "invgamma", "invweibull", "johnsonsb", "johnsonsu", "kappa4", "kappa3", "ksone", 
                  "kstwobign", "laplace", "levy", "levy_l", "logistic", "loggamma", "loglaplace",
                  "lognorm", "loguniform", "lomax", "maxwell", "mielke", "moyal", "nakagami", "ncx2","ncf",
                  "norm", "pareto", "pearson3", "powerlaw", "powerlognorm", "powernorm","rdist", 
                  "rayleigh", "rice", "semicircular", "skewnorm", "t", "trapz", "triang",
                  "truncexpon", "uniform", "vonmises", "vonmises_line", "wald", "weibull_min",
                  "weibull_max", "wrapcauchy"]
    dist_results = []
    params = {}
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        param = dist.fit(data)
        params[dist_name] = param
        D, p = st.kstest(data, dist_name, args=param)
        dist_results.append((dist_name, p))
    best_dist, best_p = (max(dist_results, key=lambda item: item[1]))
    return best_dist, best_p, params[best_dist]


dist_list = []
for col in dataset.columns:
    dist,_,_ = get_best_distribution(dataset[col])
    dist_list.append(dist)

for col,dist_ in zip(df.columns,dist_list):
    
    start_time = time.time()
    plt.hist(df[col], bins=25, density=True, alpha=0.6, color='g')

    # Plot the PDF.
    mu, std = st.norm.fit(df[col])
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, len(df))
    p = st.norm.pdf(x, mu, std)
    plt.plot(x, p,"k:", linewidth=2, label = "norm")
    
    # Plot second PDF.
    dist = getattr(st, dist_)
    param = dist.fit(df[col])
    mu1 = param[len(param)-2] 
    std1 = param[len(param)-1]
    ymin, ymax = plt.xlim()
    y = np.linspace(ymin, ymax, len(df))
    q = dist.pdf(y,*param[:-2] ,loc = mu1, scale = std1)
    plt.plot(y, q, 'r', linewidth = 1, label = dist_)
#     title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
#     plt.title(title)

    plt.legend()
    plt.savefig("images/"+col+".jpg")
    plt.clf()
    print("Time to plot:")
    print("--- %s seconds ---" % (time.time() - start_time))


Time to plot:
--- 1.2423503398895264 seconds ---
Time to plot:
--- 0.2645900249481201 seconds ---
Time to plot:
--- 1.8282182216644287 seconds ---
Time to plot:
--- 0.6971323490142822 seconds ---
Time to plot:
--- 0.8309314250946045 seconds ---
Time to plot:
--- 1.1954231262207031 seconds ---
Time to plot:
--- 0.3508119583129883 seconds ---
Time to plot:
--- 0.2493882179260254 seconds ---


<Figure size 432x288 with 0 Axes>

-----------------------------------------

In [57]:
start_point = os.getcwd()
os.chdir("TimeDistColumn/")
files = os.listdir()
path = os.getcwd()
os.chdir(start_point)

In [58]:
print(files)
print(len(files))

['time_AGE.csv', 'time_B.csv', 'time_AveRooms.csv', 'time_mean concavity.csv', 'time_ Excess kurtosis of the integrated profile.csv', 'time_worst smoothness.csv', 'time_Population.csv', 'time_mean symmetry.csv', 'time_generate column 1.csv', 'time_generate column 3.csv', 'time_od_of_di.csv', 'time_Diameter.csv', 'time_alcalinity_of_ash.csv', 'time_ Skewness of the DM-SNR curve.csv', '.ipynb_checkpoints', 'time_Viscera weight.csv', 'time_generate column 2.csv', 'time_radius error.csv', 'time_Whole weight.csv', 'time_NOX.csv', 'time_generate column 4.csv', 'time_fractal dimension error.csv', 'time_magnesium.csv', 'time_AveOccup.csv', 'time_worst concave points.csv', 'time_flavanoids.csv', 'time_mean fractal dimension.csv', 'time_alcohol.csv', 'time_worst symmetry.csv', 'time_RAD.csv', 'time_sepal.width.csv', 'time_PTRATIO.csv', 'time_mean radius.csv', 'time_Length.csv', 'time_ash.csv', 'time_mean concave points.csv', 'time_proanthocyanins.csv', 'time_symmetry error.csv', 'time_worst text

In [59]:
df = pd.read_csv("TimeDistColumn/"+files[0])
df = df.tail(15)
print(df["Distributions"])

76      crystalball
77        johnsonsb
78             burr
79             fisk
80        exponweib
81     powerlognorm
82        johnsonsu
83           kappa4
84    vonmises_line
85         vonmises
86             ncx2
87       gausshyper
88            argus
89         genexpon
90              ncf
Name: Distributions, dtype: object


In [60]:
all_files = glob.glob(path + "/*.csv")

li = []

for file in all_files:
    df = pd.read_csv(file,index_col=None, header=0)
    df = df.tail(15)
    li.append(df["Distributions"].values)

In [61]:
print(len(li))
print(li)

89
[array(['crystalball', 'johnsonsb', 'burr', 'fisk', 'exponweib',
       'powerlognorm', 'johnsonsu', 'kappa4', 'vonmises_line', 'vonmises',
       'ncx2', 'gausshyper', 'argus', 'genexpon', 'ncf'], dtype=object), array(['exponweib', 'crystalball', 'genextreme', 'burr', 'powerlognorm',
       'johnsonsb', 'vonmises_line', 'johnsonsu', 'ncx2', 'vonmises',
       'kappa4', 'argus', 'ncf', 'genexpon', 'gausshyper'], dtype=object), array(['gengamma', 'kappa3', 'johnsonsb', 'vonmises_line', 'burr',
       'argus', 'vonmises', 'powerlognorm', 'genexpon', 'kappa4', 'ksone',
       'ncx2', 'skewnorm', 'gausshyper', 'ncf'], dtype=object), array(['powernorm', 'trapz', 'johnsonsb', 'exponweib', 'burr',
       'johnsonsu', 'powerlognorm', 'kappa4', 'vonmises_line', 'ncx2',
       'vonmises', 'argus', 'gausshyper', 'ncf', 'genexpon'], dtype=object), array(['vonmises_line', 'burr12', 'fisk', 'exponweib', 'powerlognorm',
       'argus', 'vonmises', 'burr', 'kappa4', 'genexpon', 'ksone',
       'ske

In [62]:
flat = [word
        for sublist in li
        for sentence in sublist
        for word in sentence.split()
]

In [63]:
flat

['crystalball',
 'johnsonsb',
 'burr',
 'fisk',
 'exponweib',
 'powerlognorm',
 'johnsonsu',
 'kappa4',
 'vonmises_line',
 'vonmises',
 'ncx2',
 'gausshyper',
 'argus',
 'genexpon',
 'ncf',
 'exponweib',
 'crystalball',
 'genextreme',
 'burr',
 'powerlognorm',
 'johnsonsb',
 'vonmises_line',
 'johnsonsu',
 'ncx2',
 'vonmises',
 'kappa4',
 'argus',
 'ncf',
 'genexpon',
 'gausshyper',
 'gengamma',
 'kappa3',
 'johnsonsb',
 'vonmises_line',
 'burr',
 'argus',
 'vonmises',
 'powerlognorm',
 'genexpon',
 'kappa4',
 'ksone',
 'ncx2',
 'skewnorm',
 'gausshyper',
 'ncf',
 'powernorm',
 'trapz',
 'johnsonsb',
 'exponweib',
 'burr',
 'johnsonsu',
 'powerlognorm',
 'kappa4',
 'vonmises_line',
 'ncx2',
 'vonmises',
 'argus',
 'gausshyper',
 'ncf',
 'genexpon',
 'vonmises_line',
 'burr12',
 'fisk',
 'exponweib',
 'powerlognorm',
 'argus',
 'vonmises',
 'burr',
 'kappa4',
 'genexpon',
 'ksone',
 'skewnorm',
 'ncx2',
 'gausshyper',
 'ncf',
 'kstwobign',
 'trapz',
 'exponweib',
 'burr',
 'johnsonsb',


In [64]:
collections.Counter(flat)

Counter({'crystalball': 9,
         'johnsonsb': 71,
         'burr': 68,
         'fisk': 16,
         'exponweib': 79,
         'powerlognorm': 88,
         'johnsonsu': 78,
         'kappa4': 89,
         'vonmises_line': 82,
         'vonmises': 89,
         'ncx2': 84,
         'gausshyper': 89,
         'argus': 89,
         'genexpon': 89,
         'ncf': 89,
         'genextreme': 4,
         'gengamma': 4,
         'kappa3': 1,
         'ksone': 28,
         'skewnorm': 57,
         'powernorm': 27,
         'trapz': 53,
         'burr12': 8,
         'kstwobign': 22,
         'exponpow': 6,
         'halfgennorm': 1,
         'gompertz': 4,
         'triang': 7,
         'genhalflogistic': 1,
         'mielke': 1,
         'rice': 2})

In [1]:
dist_lazy = ['crystalball', 'johnsonsb', 'burr', 'fisk', 'exponweib', 'powerlognorm', 'johnsonsu',
             'kappa4', 'vonmises_line', 'vonmises', 'ncx2', 'gausshyper', 'argus', 'genexpon',
             'ncf', 'genextreme', 'gengamma', 'kappa3', 'ksone', 'skewnorm', 'powernorm', 'trapz',
             'burr12', 'kstwobign', 'exponpow', 'halfgennorm', 'gompertz', 'triang', 'genhalflogistic', 'mielke', 'rice']

In [2]:
dist_names = ["alpha", "anglit", "arcsine", "argus", "beta", "betaprime", "bradford", "burr", "burr12", "cauchy", "chi",
                    "chi2","cosine","crystalball", "dgamma", "dweibull", "erlang", "expon", 
                    "exponnorm","exponweib", "exponpow", "f", "fatiguelife", "fisk", "foldcauchy", "foldnorm", 
                    "frechet_r", "frechet_l", "genlogistic", "gennorm", "genpareto", "genexpon", "genextreme",
                    "gausshyper", "gamma", "gengamma", "genhalflogistic", "gilbrat", "gompertz", 
                    "gumbel_r", "gumbel_l", "halfcauchy", "halflogistic", "halfnorm", "halfgennorm", "hypsecant", 
                    "invgamma", "invweibull", "johnsonsb", "johnsonsu", "kappa4", "kappa3", "ksone", 
                    "kstwobign", "laplace", "levy", "levy_l", "logistic", "loggamma", "loglaplace",
                    "lognorm", "loguniform", "lomax", "maxwell", "mielke", "moyal", "nakagami", "ncx2","ncf",
                    "norm", "pareto", "pearson3", "powerlaw", "powerlognorm", "powernorm","rdist", 
                    "rayleigh", "rice", "semicircular", "skewnorm", "t", "trapz", "triang",
                    "truncexpon", "uniform", "vonmises", "vonmises_line", "wald", "weibull_min",
                    "weibull_max", "wrapcauchy"]
print(len(dist_names))

91


In [3]:
li = set(dist_names) - set(dist_lazy)

In [4]:
li

{'alpha',
 'anglit',
 'arcsine',
 'beta',
 'betaprime',
 'bradford',
 'cauchy',
 'chi',
 'chi2',
 'cosine',
 'dgamma',
 'dweibull',
 'erlang',
 'expon',
 'exponnorm',
 'f',
 'fatiguelife',
 'foldcauchy',
 'foldnorm',
 'frechet_l',
 'frechet_r',
 'gamma',
 'genlogistic',
 'gennorm',
 'genpareto',
 'gilbrat',
 'gumbel_l',
 'gumbel_r',
 'halfcauchy',
 'halflogistic',
 'halfnorm',
 'hypsecant',
 'invgamma',
 'invweibull',
 'laplace',
 'levy',
 'levy_l',
 'loggamma',
 'logistic',
 'loglaplace',
 'lognorm',
 'loguniform',
 'lomax',
 'maxwell',
 'moyal',
 'nakagami',
 'norm',
 'pareto',
 'pearson3',
 'powerlaw',
 'rayleigh',
 'rdist',
 'semicircular',
 't',
 'truncexpon',
 'uniform',
 'wald',
 'weibull_max',
 'weibull_min',
 'wrapcauchy'}